In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.stats import linregress
from scipy.optimize import fsolve
from scipy.interpolate import UnivariateSpline
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime
from scipy.optimize import curve_fit
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
N = 8
w0 = 0.2
w = 1.8
alpha = 100
h = 0.001
wEE = 0
wEI = 0
wIE = 0
wII = 0
hE = h
hI = h

In [ ]:
def get_W (N, w, w0, frac_inhib):
    NI = int(N*frac_inhib)
    NE = N - NI

    wE = ((w+w0)/2)
    wI = (wE-w0)

    wE = wE/NE
    wI = (-1)*wI/NI
    W = np.zeros((N, N))

    # Seleccionar aleatoriamente NI índices únicos para neuronas inhibitorias
    is_inhibitory = np.random.choice(N, NI, replace=False)

    # Las restantes serán neuronas excitatorias
    is_excitatory = np.setdiff1d(np.arange(N), is_inhibitory)

    for i in range(N):
        for j in range(N):
            if i!=j:
                if i in is_inhibitory:
                    W[i][j] = wI
                else:
                    W[i][j] = wE

    return W,is_excitatory, is_inhibitory, NE, NI

In [ ]:
def f(x):
    return np.where(x > 0, np.tanh(x), 0)


# # Función de activación
# def f(s):
#     return np.tanh(s) * (s > 0)


In [ ]:
# Derivadas deterministas
def dE_dt(E, I):
    sE = wEE * E - wEI * I + hE
    return -alpha * E + (1 - E) * f(sE)

def dI_dt(E, I):
    sI = wIE * E - wII * I + hI
    return -alpha * I + (1 - I) * f(sI)


In [ ]:
def get_estado_inicial(Red, frac_active_NE=0.5, frac_active_NI=0.5, NE=0, NI=0, excit_indices= [], inhib_indices= []):

    NE_activos = int(frac_active_NE * NE)
    NI_activos = int(frac_active_NI * NI)

    # Convertir a listas para usar con random.sample
    inhib_indices = list(inhib_indices)
    excit_indices = list(excit_indices)
    
    # Seleccionar índices aleatorios sin reemplazo usando numpy
    inhib_seleccionados = np.random.choice(inhib_indices, size=NI_activos, replace=False)
    excit_seleccionados = np.random.choice(excit_indices, size=NE_activos, replace=False)

    indices_a_activar = np.concatenate([inhib_seleccionados, excit_seleccionados]).astype(np.int32)

    # Activar los elementos seleccionados
    Red[indices_a_activar] = 1

    return Red


In [ ]:
def gillespie_algorithm(Red, W, T, log=0):
    tiempo_inicio = time.time()

    t = 0
    times = [t]
    rates = np.zeros(N)
    States = []
    Rates = []
    Spikes = []
    
    States.append(Red.copy())
    Rates.append(np.zeros(N))
    Spikes.append(np.zeros(N))

    step = 0
    while t <= T:
        spikes = np.zeros(N)
        #Calculamos los rates       
        r = 0
        for i in range(N):
            if Red[i] == 0: #=> La neurona está apagada f(s_i)
                suma = h
                for j in range(N):
                    suma += Red[j]*W[j][i]
                suma = f(suma)
                rates[i] = suma
                r += suma
            else: #=> La neurona está encendida tiene un CD de alpha segundos
                rates[i] = alpha
                r += alpha

        #Cuál debe ser el paso para que ocurra una "reacción"
        u1, u2 = np.random.uniform(0, 1, 2)
        
        dt = (1/r)*np.log(1/u1)
        #Qué neurona es la que sufre esa reacción?
        suma = 0
        i = 0
        sigo = True
        while i < N and sigo:
            suma += rates[i] / r
            if u2 <= suma:
                neurona = i
                sigo = False
            else:
                i += 1
        
        #Cambio el estado de dicha neurona
        if Red[neurona] == 0:
            Red[neurona] = 1
            spikes[neurona] = 1
        elif Red[neurona] == 1:
            Red[neurona] = 0

        #Avanzo el tiempo
        t += dt
        if log > 0:
            if (step % log == 0): 
                tiempo_actual = time.time()
                tiempo_transcurrido = tiempo_actual - tiempo_inicio
                print(
                    f"\rSe han realizado {step+1} pasos | t={t:.3f}s ({t/T*100:.2f}%) | "
                    f"Tiempo transcurrido: {tiempo_transcurrido:.2f}s | ETA: {tiempo_transcurrido/(t/T):.2f}s",
                    end="",
                    flush=True
                )


        step += 1
        States.append(Red.copy())
        Rates.append(rates.copy())
        Spikes.append(spikes.copy())
        times.append(t)

    
    print()
    return np.array(States), np.array(Rates), np.array(Spikes), np.array(times)


In [ ]:
def get_ISI(times, Spikes):
    times = np.array(times)
    Spikes = np.array(Spikes)  # Asegúrate de que es un array NumPy
    
    # Si Spikes es 2D (n_steps x n_neurons), busca cualquier spike por fila
    if Spikes.ndim == 2:
        mask = np.any(Spikes > 0, axis=1)
    else:  # Si es 1D
        mask = Spikes > 0

    spike_times = times[mask]
    isis = np.diff(spike_times)
    
    if len(isis) == 0:
        return None  # o np.nan o lanzar un error
    return np.mean(isis)



In [ ]:
def get_avalanche_distribution(times, Spikes):
    times = np.array(times)
    Spikes = np.array(Spikes)
    
    # 1. Detectar tiempos de spike
    if Spikes.ndim == 2:
        mask = np.any(Spikes > 0, axis=1)
    else:
        mask = Spikes > 0

    spike_times = times[mask]
    
    # Si no hay spikes suficientes, devolver lista vacía
    if len(spike_times) < 2:
        return []

    # 2. Calcular ISI promedio
    isi_avg = np.mean(np.diff(spike_times))
    
    # 3. Binarizar el tiempo en bins de tamaño ISI_avg
    t_min, t_max = times[0], times[-1]
    bins = np.arange(t_min, t_max + isi_avg, isi_avg)
    binned_counts, _ = np.histogram(spike_times, bins=bins)

    # 4. Detectar avalanchas: secuencias de bins con spikes (>0) separados por 0s
    avalanches = []
    size = 0
    for count in binned_counts:
        if count > 0:
            size += count
        elif size > 0:
            avalanches.append(size)
            size = 0
    # Si termina con una avalancha activa
    if size > 0:
        avalanches.append(size)

    return avalanches


In [ ]:
def analyze_avalanche_distribution(times, Spikes):
    avalanches = get_avalanche_distribution(times, Spikes)

    if len(avalanches) == 0:
        print("No se detectaron avalanchas.")
        return

    # Histograma logarítmico
    bins = np.logspace(np.log10(1), np.log10(max(avalanches)+1), num=20)
    counts, edges = np.histogram(avalanches, bins=bins, density=True)

    bin_centers = (edges[:-1] + edges[1:]) / 2

    mask = counts > 0
    x = bin_centers[mask]
    y = counts[mask]
    y = y / np.sum(y)

    # Ajuste lineal en escala log-log
    log_x = np.log10(x)
    log_y = np.log10(y)
    slope, intercept, r_value, p_value, std_err = linregress(log_x, log_y)

    alpha = -slope  # ley de potencias: P(x) ~ x^(-alpha)

    # Graficar
    plt.figure(figsize=(6, 4))
    plt.scatter(x, y, color='black', label='Datos')
    plt.plot(x, 10**(intercept + slope * np.log10(x)), label=f'Ajuste (α ≈ {alpha:.2f})', linestyle='--')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Tamaño de avalancha')
    plt.ylabel('Densidad de probabilidad')
    plt.title('Distribución de tamaño de avalanchas')
    plt.grid(True, which='both', ls='--', linewidth=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"Ajuste ley de potencias (estimación log-log): alpha ≈ {alpha:.2f}")
    print(f"R² del ajuste: {r_value**2:.3f}")


In [ ]:
#Parámetros del sistema
w_param = [0.8, 1.8, 13.8]

for i in range(1):
    N = 10000
    frac_inhib = 0.5
    w = w_param[i]
    w0 = 0.2

    W, excit_indices, inhib_indices, NE, NI = get_W(N, w, w0, frac_inhib=frac_inhib)
    Red = get_estado_inicial(np.zeros(N), frac_active_NE=0.5, frac_active_NI=0.5, NE=NE, NI=NI, inhib_indices=inhib_indices, excit_indices=excit_indices)

    #Parámetros de la simulación
    T = 0.1

    alpha = 0.1 #esto me imagino que lo tengo que pasar a segundos tau = 1ms + 9ms = 10ms
    h = 0.001

    States, Rates, Spikes, times= gillespie_algorithm(Red, W, T, log=100)


    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    # Para cada simulación
    # np.savez_compressed(
    #     f"../Resultados/sim_{timestamp}.npz",
    #     W=W,
    #     NE=NE,
    #     NI=NI,
    #     excit_indices=excit_indices, 
    #     inhib_indices=inhib_indices,
    #     States=States,
    #     Rates=Rates,
    #     Spikes=Spikes,
    #     Times=times,
    #     N=N,
    #     w=w,
    #     w0=w0,
    #     alpha=alpha,
    #     h=h
    # )


In [ ]:
data = np.load("../Resultados/sim_20250707_191320.npz")
States = data["States"]
Rates = data["Rates"]
Spikes = data["Spikes"]
times = data["Times"]
N = int(data["N"])
w = float(data["w"])
w0 = float(data["w0"])
alpha = float(data["alpha"])
h = float(data["h"])
excit_indices = data["excit_indices"]
inhib_indices = data["inhib_indices"]

In [ ]:
all_indices = np.arange(N)
inhib_indices = np.array(inhib_indices)
excit_indices = np.setdiff1d(all_indices, inhib_indices)

# Reordenamos usando bucles
States_ordered = []
Spikes_ordered = []
Rates_ordered = []

for t in range(len(States)):
    s_ordered = np.concatenate([States[t][inhib_indices], States[t][excit_indices]])
    sp_ordered = np.concatenate([Spikes[t][inhib_indices], Spikes[t][excit_indices]])
    r_ordered = np.concatenate([Rates[t][inhib_indices], Rates[t][excit_indices]])
    
    States_ordered.append(s_ordered)
    Spikes_ordered.append(sp_ordered)
    Rates_ordered.append(r_ordered)

# Convertimos a arrays para graficar
States_ordered = np.array(States_ordered)
Spikes_ordered = np.array(Spikes_ordered)
Rates_ordered = np.array(Rates_ordered)


mean_firing = np.mean(Rates, axis=1)
std_firing = np.std(Rates, axis=1)

In [ ]:
# Dibujamos las 3 matrices una debajo de otra
fig, axes = plt.subplots(3, 1, figsize=(8, 6), sharex=True)

axes[0].imshow(np.transpose(States_ordered), cmap='gray_r', aspect='auto')
axes[0].set_title("States")

axes[1].imshow(np.transpose(Spikes_ordered), cmap='gray_r', aspect='auto')
axes[1].set_title("Spikes")

axes[2].imshow(np.transpose(Rates_ordered), cmap='gray_r', aspect='auto')
axes[2].set_title("Rates")
axes[2].set_xlabel("Time")

plt.tight_layout()
plt.show()

In [ ]:
analyze_avalanche_distribution(times, Spikes)

In [ ]:
#Evolución del sistema
NI_evol = []
NE_evol = []

for step in States:
    NI_active = sum(step[inhib_indices])
    NE_active = sum(step[excit_indices])
    NI_evol.append(NI_active/N*2)
    NE_evol.append(NE_active/N*2)

# Parámetros del modelo
WE = (w+w0)/2
WI = WE-w0
wEE = WE
wEI = WI
wIE = WE
wII = WI
hE = h
hI = h

#Campo vectorial y las nulclinas
E_field_vals = np.linspace(0, 1, 10)
I_field_vals = np.linspace(0, 1, 10)
E_field, I_field = np.meshgrid(E_field_vals, I_field_vals)

dE = dE_dt(E_field, I_field)
dI = dI_dt(E_field, I_field)

# Normalizar vectores
magnitude = np.sqrt(dE**2 + dI**2)
dE_norm = dE*3
dI_norm = dI*3

E_contour_vals = np.linspace(0, 1, 50)
I_contour_vals = np.linspace(0, 1, 50)
E_contour, I_contour = np.meshgrid(E_contour_vals, I_contour_vals)

dE_contour = dE_dt(E_contour, I_contour)
dI_contour = dI_dt(E_contour, I_contour)

plt.figure(figsize=(7, 6))

# Campo vectorial
plt.quiver(E_field, I_field, dE_norm, dI_norm, color='black', alpha=1, width=0.003, scale=40)

# Nullclines
cs1 = plt.contour(E_contour, I_contour, dE_contour, levels=[0], colors='red', linewidths=2, zorder=20)
cs2 = plt.contour(E_contour, I_contour, dI_contour, levels=[0], colors='blue', linewidths=2, zorder=20)
# plt.clabel(cs1, fmt={'0': '$dE/dt=0$'}, colors='red')
# plt.clabel(cs2, fmt={'0': '$dI/dt=0$'}, colors='blue')

# # Punto fijo
# plt.scatter(
#     E_fp, 
#     I_fp, 
#     facecolor=(1, 1, 1, 0), 
#     edgecolors=(0, 0, 0), 
#     label='Punto fijo',
#     s=50,
#     zorder=15)

plt.plot(NE_evol, NI_evol, color=(0, 1, 0), label='Trayectoria')
plt.scatter(
    NE_evol[-1], 
    NI_evol[-1],
    facecolor=(1, 1, 1, 0), 
    edgecolors=(1, 0, 0), 
    label='Punto final',
    s=50,
    zorder=10)

plt.xlabel("E (actividad excitatoria)")
plt.ylabel("I (actividad inhibitoria)")
plt.title(fr"w={w}, w0={w0}, h={h}, $\alpha$={alpha}")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.gca().set_aspect('equal', adjustable='box')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Crear malla fina para el fondo
E_bg, I_bg = np.meshgrid(np.linspace(0, 1, 300), np.linspace(0, 1, 300))
# Definir el gradiente como interpolación lineal entre dos esquinas
gradient = -np.arctan(5*(E_bg - I_bg))
# Crear el mapa de color personalizado
cmap = LinearSegmentedColormap.from_list('strong_gradient', ['blue','blue', 'red', 'red'])
# Definimos la función de activación (sigmoide tipo tanh truncada)
def f(s):
    return np.where(s >= 0, np.tanh(s), 0)

# Parámetros del modelo
alpha = 0.1
w_EE = 1
w_EI = 0.5
w_IE = 1
w_II = 0.5
h_E = 0.001
h_I = 0.001

# Campo vectorial de las ecuaciones de Wilson-Cowan
def dE_dt(E, I):
    s_E = w_EE * E - w_EI * I + h_E
    return -alpha * E + (1 - E) * f(s_E)

def dI_dt(E, I):
    s_I = w_IE * E - w_II * I + h_I
    return -alpha * I + (1 - I) * f(s_I)

# Definir sistema de ecuaciones para encontrar el punto fijo (intersección de nulclinas)
def system(vars):
    E, I = vars
    eq1 = dE_dt(E, I)
    eq2 = dI_dt(E, I)
    return [eq1, eq2]


# Crear una malla de puntos en el espacio E-I
E_vals = np.linspace(0, 1, 20)
I_vals = np.linspace(0, 1, 20)
E, I = np.meshgrid(E_vals, I_vals)
dE = dE_dt(E, I)
dI = dI_dt(E, I)


# Usar fsolve para encontrar el punto fijo
initial_guess = [0.5, 0.5]
E_fp, I_fp = fsolve(system, initial_guess)

# Calcular nulclinas: puntos donde dE/dt = 0 y dI/dt = 0
E_nc = np.linspace(0, 1, 1000)
I_nc = np.linspace(0, 1, 1000)
EE, II = np.meshgrid(E_nc, I_nc)
nullcline_E = dE_dt(EE, II)
nullcline_I = dI_dt(EE, II)
# Recalcular las nulclinas como listas separadas sin usar tuplas
E_null_E = []
I_null_E = []
E_null_I = []
I_null_I = []

for e in E_nc:
    i_e = fsolve(lambda i: dE_dt(e, i), 0.00001)[0]
    if 0 <= i_e <= 1:
        E_null_E.append(e)
        I_null_E.append(i_e)

    i_i = fsolve(lambda i: dI_dt(e, i), 0.00001)[0]
    if 0 <= i_i <= 1:
        E_null_I.append(e)
        I_null_I.append(i_i)


In [ ]:
plt.plot(times, NI_evol, label="NI", color="blue")
plt.plot(times, NE_evol, label="NE", color="red")
plt.xlabel("time (s)")
plt.grid()
plt.ylabel("% activation")
plt.ylim(0, 1)
plt.xlim(0, T)
plt.title("Evolución de la actividad por tipo de neurona")
plt.legend()

In [ ]:
def get_branching_data(times, Spikes):
    times = np.array(times)
    Spikes = np.array(Spikes)

    if Spikes.ndim != 2:
        raise ValueError("Spikes debe ser un array 2D (tiempo x canales)")

    # 1. Detectar actividad
    active = np.any(Spikes > 0, axis=1)
    spike_times = times[active]

    if len(spike_times) < 2:
        return [], []

    isi_avg = np.mean(np.diff(spike_times))

    # 2. Binarizar en bins temporales
    t_min, t_max = times[0], times[-1]
    bins = np.arange(t_min, t_max + isi_avg, isi_avg)
    digitized = np.digitize(times, bins) - 1

    # 3. Contar cuántos electrodos activos hay por bin temporal
    binned_activity = np.zeros((len(bins)-1, Spikes.shape[1]), dtype=int)
    for i in range(len(times)):
        if 0 <= digitized[i] < len(binned_activity):
            binned_activity[digitized[i]] |= (Spikes[i] > 0)

    binned_counts = np.sum(binned_activity, axis=1)

    # 4. Detectar avalanchas y recolectar na y nd
    na_list = []
    nd_list = []
    i = 0
    while i < len(binned_counts) - 1:
        if binned_counts[i] > 0:
            na = binned_counts[i]
            nd = binned_counts[i + 1]
            na_list.append(na)
            nd_list.append(nd)
            # avanzar hasta terminar avalancha
            while i < len(binned_counts) and binned_counts[i] > 0:
                i += 1
        else:
            i += 1

    return na_list, nd_list


In [ ]:
import numpy as np

def compute_branching_parameter(ancestor_counts, descendant_counts, nmax):
    assert len(ancestor_counts) == len(descendant_counts), 
    
    p_d = {}
    total_ancestors = 0
    
    for na, nd in zip(ancestor_counts, descendant_counts):
        if na == 0:
            continue
        d = round(nd / na)
        key = d
        correction_factor = (nmax - 1) / (nmax - na) if (nmax - na) != 0 else 0
        if key in p_d:
            p_d[key]['na_sum'] += na * correction_factor
            p_d[key]['count'] += 1
        else:
            p_d[key] = {'na_sum': na * correction_factor, 'count': 1}
        total_ancestors += na

    sigma = sum([d * (entry['na_sum'] / total_ancestors) for d, entry in p_d.items()])
    return sigma



In [ ]:
na, nd = get_branching_data(times, Spikes)
sigma = compute_branching_parameter(na, nd, nmax=Spikes.shape[1])
print(f"Parámetro de branching estimado: σ ≈ {sigma:.3f}")
